In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision as models
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch.nn.functional as F
from datetime import datetime

plt.ion()   # interactive mode

In [2]:
lr = 0.001
epochs = 10
bs = 256

# dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
cpu = torch.device("cpu")
device = torch.device("cuda")

In [3]:
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
datadir = "data"
train_dir = "train"
val_dir = "val"

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=bs,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    
    train_acc = []
    val_acc = []
    
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if epoch % 10 == 0:
                print(datetime.now(), 'Phase: {} Epoch: {}/{} Loss: {:.4f} Acc: {:.4f}'
                      .format(phase, epoch+1, num_epochs, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [5]:
loss_func = F.cross_entropy

In [6]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [7]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('A:{} P:{}'.format(class_names[labels[j]], class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [8]:
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
# vgg16 = models.vgg16(pretrained=True) # too large
# densenet = models.densenet161(pretrained=True) # too large
# inception = models.inception_v3(pretrained=True) # too large
# googlenet = models.googlenet(pretrained=True) # too large
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
# mobilenet = models.mobilenet_v2(pretrained=True) # too large
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True) # too large
# wide_resnet50_2 = models.wide_resnet50_2(pretrained=True) # too large
# mnasnet = models.mnasnet1_0(pretrained=True) # too large
resnet34 = models.resnet34(pretrained=True)
resnet50 = models.resnet50(pretrained=True)
resnet101 = models.resnet101(pretrained=True)
resnet152 = models.resnet152(pretrained=True)

In [9]:
# TODO: How to select a pretrained model for your problem.
#       Describe the process to adapt the pretrained model for your problem.

def adapt_model(model_name):
    if model_name == "resnet18":
        model = models.resnet18(pretrained=True)
        fc_in_features = model.fc.in_features
        model.fc = nn.Linear(fc_in_features, 120)
    elif model_name == "resnet34":
        model = models.resnet34(pretrained=True)
        fc_in_features = model.fc.in_features
        model.fc = nn.Linear(fc_in_features, 120)
    elif model_name == "resnet50":
        model = models.resnet50(pretrained=True)
        fc_in_features = model.fc.in_features
        model.fc = nn.Linear(fc_in_features, 120)
    elif model_name == "resnet101":
        model = models.resnet101(pretrained=True)
        fc_in_features = model.wwwwwsaaaaafc.in_features
        model.fc = nn.Linear(fc_in_features, 120)
    elif model_name == "resnet152":
        model = models.resnet152(pretrained=True)
        fc_in_features = model.fc.in_features
        model.fc = nn.Linear(fc_in_features, 120)
    elif model_name == "shufflenet":
        model = models.shufflenet_v2_x1_0(pretrained=True)
        fc_in_features = model.fc.in_features
        model.fc = nn.Linear(fc_in_features, 120)
    elif model_name == "alexnet":
        modle = models.alexnet(pretrained=True)
        fc_in_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(fc_in_features, 120)
    elif model_name == "squeezenet":
        model = models.squeezenet1_0(pretrained=True)
        old_conv_layer = model.classifier[1]
        model.classifier[1] = nn.Conv2d(old_conv_layer.in_channels, 120, kernel_size=old_conv_layer.kernel_size, 
                                        stride=old_conv_layer.kernel_size)
    return model

In [10]:
#plt.rcParams["figure.figsize"] = (20,20)
#visualize_model(model_ft)

In [ ]:
# resnet18 best for 10 epochs
# resnet34 best for 100 epochs
pretrained = ["resnet34", "resnet50", "resnet101", "resnet152", "resnet18", "alexnet", "squeezenet", "shufflenet"]
print("Starting to train models: CUDA mem:", torch.cuda.memory_allocated())
for model_name in pretrained:
    print("Training model:", model_name, "CUDA mem:", torch.cuda.memory_allocated())
    model = adapt_model(model_name)
    model.to(device)
    opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(opt, step_size=7, gamma=0.1)
    model_ft = train_model(model, loss_func, opt, exp_lr_scheduler, num_epochs=epochs)
    del model_ft
    del exp_lr_scheduler
    del opt
    torch.cuda.empty_cache()

Starting to train models: CUDA mem: 0
Training model: resnet34 CUDA mem: 0
2020-04-05 13:22:56.808421 Phase: train Epoch: 1/10 Loss: 4.6681 Acc: 0.0487
2020-04-05 13:23:24.108767 Phase: val Epoch: 1/10 Loss: 4.0698 Acc: 0.1793
